In [ ]:
import pandas as pd
import numpy as np
import base64
import pyodbc
import os

In [ ]:
# Create Connection
password = config.password
# Connecting to SQL Server
client_code = '544'
database = 'SS'
base_dir = r"C:\Users\jesse.henson\StaffScapes\StaffScapes - Documents\IT\Isolved Import\EE_upload"
conn = pyodbc.connect('Driver={SQL Server Native Client 10.0};'
                      'Server=staffc-darwin\darwin12;'
                      f'Database={database};'
                      'UID=username;'
                      'PWD=' + password + ';')
if not os.path.isdir(f'{base_dir}/{client_code}'):
    os.mkdir(f'{base_dir}/{client_code}')
# paycode_data = pd.read_sql_query('Select * from upr30300',conn,parse_dates=['PeriodBeginDate','PeriodEndDate','CheckDate'])
# paycode_data

In [ ]:

sql_str = '''
select format(upr00100.STRTDATE,'MM/dd/yyyy') as AdjustedServiceDate, 
upr00100.EMPLOYID as EmployeeNumber, 
upr00100.FRSTNAME as FirstName, 
upr00100.LASTNAME as LastName, 
TW00824.DefaultEmailAddress AS PersonalEmail,
TW00824.DefaultEmailAddress AS SelfServiceEmail,
upr00102.ADDRESS1 as Address1, 
upr00102.ADDRESS2 as Address2,
upr00102.ZIPCODE as ZipCode, 
upr00102.CITY as City, 
upr00102.state as State, 
'26' as Frequency, 
EMPLOYMENTTYPE as EmploymentCategory, 
upr00100.DEMPINAC as StatusDate,
'Corp' as WorkLocation,
ZIPCODE as ResidentLocation,
Deprtmnt as #department,
UPR00100.SOCSCNUM AS SSN,
'SSN' AS IDType,
format(tw00820.ThinkwareOriginHireDate, 'MM/dd/yyy') as HireDate,
upr00100.BRTHDATE as BirthDate,
upr00102.PHONE1 as HomePhone,
upr00100.ETHNORGN as EthnicOrigin,
upr00100.GENDER as Gender,
upr00100.WRKRCOMP as WorkersComp,
upr00100.INACTIVE as Status
from upr00100
join tw00819 on upr00100.EMPLOYID = tw00819.EMPLOYID
left outer join tw00820 on upr00100.employid = tw00820.employid
left outer join upr00102 on upr00100.EMPLOYID = upr00102.EMPLOYID and upr00100.ADRSCODE = upr00102.ADRSCODE
left outer JOIN TW00824 ON UPR00100.EMPLOYID = TW00824.EMPLOYID
'''

where_claus = f"where CUSTNMBR = '{client_code}'"

employee_main = pd.read_sql_query(sql_str + where_claus,conn, parse_dates=['BirthDate'])
employee_main['BirthDate'] = employee_main['BirthDate'].dt.strftime('%m/%d/%Y')
employee_main['Gender'] = employee_main['Gender'].map({1:'M',2:'F'} )
employee_main['EthnicOrigin'] = employee_main['EthnicOrigin'].map({1:2, 7:8, 5:1, 6:7, 3:3, 2:6, 8:4, 4:5})
employee_main['EthnicOrigin'] = employee_main['EthnicOrigin'].fillna(7)
employee_main['EthnicOrigin'] = employee_main['EthnicOrigin'].apply(lambda x: int(x))
employee_main['EmployeeNumber'] = employee_main['EmployeeNumber'].apply(lambda x:x[2:10])
employee_main['EmploymentCategory'] = employee_main['EmploymentCategory'].map({0: 'FT', 1: 'FT', 2: 'PT', 3:'FT-T', 4:'PT-T'})
employee_main['Status'] = employee_main['Status'].map({0: 'A', 1: 'T'})
employee_main['HomePhone'] = employee_main['HomePhone'].apply(lambda x:x[0:10])
# employee_main['SelfServiceEnabled'] = employee_main['Status'].apply(lambda x: "Y" if x== 'A' else '')
employee_main['WorkersComp'] = employee_main['WorkersComp'].apply(lambda x: x[2:])
# employee_main['EmploymentCategory'] = employee_main['EmploymentCategory'].apply(lambda x:'FT' if x == 'FT-T' else 'PT')
employee_main['EmploymentCategory']

In [ ]:
sql_str = '''
SELECT EMPLOYID as EmployeeNumber, PAYRCORD as PayType, PAYRTAMT as HourlyRate FROM upr00400
'''


where_claus = f"where PAYRCORD in ('Salary','hourly') and EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999'"

rate_main = pd.read_sql_query(sql_str + where_claus,conn)
rate_main['EmployeeNumber'] = rate_main['EmployeeNumber'].apply(lambda x:x[2:10])
rate_main['PerPaySalary'] = rate_main['HourlyRate'].apply(lambda x: x if x > 1000 else 0 )
rate_main['HourlyRate'] = rate_main['HourlyRate'].apply(lambda x: 0 if x > 1000 else x)
rate_main

In [ ]:
sql_str = '''
SELECT
upr00300.EMPLOYID as EmployeeNumber, 
UPR00300.FDFLGSTS AS FedFilingStatus,
upr00300.FEDEXMPT as FedExemptions,
upr00300.ADFDWHDG as FedAddlAmount,
TWUPR003.DependentClaimAmt as FedDependentsAmt,
upr00700.TXFLGSTS as StateFilingStatus,
upr00700.EXMTAMNT as StateExemptionAmount,
upr00700.ADSTWHDG as StateAddlAmount,
upr00700.ADNLALOW as StateAddlExemptions FROM UPR00300
JOIN TWUPR003 ON UPR00300.EMPLOYID = TWUPR003.EMPLOYID
join upr00700 on upr00300.EMPLOYID = upr00700.EMPLOYID
'''

where_claus = f"where upr00300.EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999'"


tax_main = pd.read_sql_query(sql_str + where_claus, conn)
tax_main['EmployeeNumber'] = tax_main['EmployeeNumber'].apply(lambda x:x[2:10])
tax_main['FedFilingStatus'] = tax_main['FedFilingStatus'].apply(lambda x: 'FDM' if x.strip() == 'MAR' else 'FDS')
tax_main['StateFilingStatus'] = tax_main['StateFilingStatus'].apply(lambda x: 'COM' if x.strip() == 'MAR' else 'COS')
tax_main['StateAddlAmount'] = tax_main['StateAddlAmount'].apply(lambda x: 0 if x < 0 else x)
tax_main

In [ ]:
temp_df = employee_main.merge(rate_main, 'left', on='EmployeeNumber')
ee_full = temp_df.merge(tax_main, 'left', on='EmployeeNumber')
ee_full['PayType'] = ee_full['PayType'].fillna('HOURLY')
ee_full['PayType'] = ee_full['PayType'].replace('','HOURLY')
ee_full['PayType'] = ee_full['PayType'].apply(lambda x: 'HOURLY' if str(x).strip() == '' else x)
ee_full.to_csv(f'{base_dir}/{client_code}/ee_full.csv')
ee_full['PayType']

In [ ]:

sql_str = '''
select tw00107.EMPLOYID AS [Key], ThinkInactive as ActiveStatus, ThinkPrecheck as PrecheckStatus, ThkAcctDesc as AccountType, ThinkDirDepOpt as Sequence, 'Y' as Frequency, 
ThinkwarePerc as [Percent], ThkDirDepAmt as Amount, ThkAcctNo as AccountNumber, ThinkBanks as RoutingNumber
from TW00107
JOIN UPR00100 on upr00100.employid = tw00107.employid
'''

where_claus = f"where tw00107.EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999' and upr00100.inactive = 0"

dd_main = pd.read_sql_query(sql_str + where_claus, conn)
dd_main['Key'] = dd_main['Key'].apply(lambda x:x[2:10])
dd_main['AccountType'] = dd_main['AccountType'].apply(lambda x: 'C' if x.strip() == 'CHECKING' else 'S')
dd_main['Status'] = dd_main['ActiveStatus'] * 10 + dd_main['PrecheckStatus'] * 5
dd_main['Status'] = dd_main['Status'].map({0:'A', 5:'P', 10:'I', 15:'I'})
dd_main['Sequence'] = dd_main['Sequence'].apply(lambda x: 0 if x==4 else 1)
dd_main['Amount'] = dd_main['Amount'].replace(0,'')
dd_main['Percent'] = dd_main['Percent'].replace(0,'')
dd_main.to_csv(f'{base_dir}/{client_code}/dd_main.csv')
dd_main

In [ ]:
sql_str = '''
select EMPLOYID as [Key], BENEFIT as PayItem, BNFPRCNT_1 as [Percent], BNFITAMT_1 as Amount from upr00600
'''

where_claus = f"where EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999' and Inactive = 0"

ben_main = pd.read_sql_query(sql_str + where_claus, conn)
ben_main['Key'] = ben_main['Key'].apply(lambda x:x[2:10])
ben_main = ben_main[ben_main['PayItem'].str.contains('TAX') == False]
ben_main = ben_main[ben_main['PayItem'].str.contains('401') == False]
ben_main['ParamType'] = ben_main['Percent'].apply(lambda x: "Percent" if x > 0 else "Amount")
ben_main['ParamValue'] = ben_main['Percent'] + ben_main['Amount']
ben_main.to_csv(f'{base_dir}/{client_code}/ben_main.csv')
ben_main

In [ ]:
sql_str = '''
select EMPLOYID as [Key], DEDUCTON as PayItem, DEDNPRCT_1 as [Percent], DEDCAMNT_1 as Amount from upr00500
'''

where_claus = f"where EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999' and Inactive = 0"

ded_main = pd.read_sql_query(sql_str + where_claus, conn)
ded_main['Key'] = ded_main['Key'].apply(lambda x:x[2:10])
ded_main = ded_main[ded_main['PayItem'].str.contains('TAX') == False]
ded_main = ded_main[ded_main['PayItem'].str.contains('401') == False]
ded_main = ded_main[ded_main['PayItem'].str.contains('SIM') == False]
ded_main = ded_main[ded_main['PayItem'].str.contains('CHILD') == False]
ded_main = ded_main[ded_main['PayItem'].str.contains('GRN') == False]
ded_main['ParamType'] = ded_main['Percent'].apply(lambda x: "Percent" if x > 0 else "Amount")
ded_main['ParamValue'] = ded_main['Percent'] + ded_main['Amount']
ded_main.to_csv(f'{base_dir}/{client_code}/ded_main.csv')
ded_main

In [ ]:
sql_str = '''
select right(rtrim(EMPLOYID), 7) as [Key], DSCRIPTN as [Plan], TW_AvailableHours as Balance from tw80514
'''

where_claus = f"where EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999' and Inactive = 0"
acc_main = pd.read_sql_query(sql_str + where_claus, conn)
acc_main = pd.read_sql_query(sql_str + where_claus, conn)
acc_main['Balance'] = acc_main['Balance'] * .01
acc_main.to_csv(f'{base_dir}/{client_code}/accrual_main.csv')
acc_main